## 2.1.1 Transformer原理
Transformer 是一种基于自注意力机制的神经网络架构，由谷歌于2017年在论文[《Attention Is All You Need》](https://arxiv.org/pdf/1706.03762)中提出。Transformer模型摒弃了固有的定式，并没有用任何CNN或者RNN的结构，而是使用了Attention注意力机制，自动捕捉输入序列不同位置处的相对关联，善于处理较长文本，并且该模型可以高度并行地工作，训练速度很快。

### **Transformer整体结构**

<center>
<img src='./images/transformer.png' height='600'/>

Transformer模型结构图
</center>

Transformer 由编码器和解码器组成，每层包含以下模块：
- **编码器层**：
  1. 多头自注意力机制 + 残差连接 + 归一化。
  2. 前馈全连接层 + 残差连接 + 归一化。
- **解码器层**：
  1. 掩码多头自注意力机制 + 残差连接 + 归一化。
  2. 多头注意力机制（编码器-解码器注意力） + 残差连接 + 归一化。
  3. 前馈全连接层 + 残差连接 + 归一化。

#### **Embedding 层**
Embedding 层将输入文本中的单词转换成向量表示，即词嵌入（word embeddings）。这些向量捕捉了词汇表中每个单词的语义信息。在Transformer中，通常使用预训练好的词向量，如Word2Vec或GloVe，或者是在训练过程中学习得到的词嵌入。

- **输入**：一个符号序列（如单词序列），每个符号用整数索引表示。
- **输出**：每个符号对应的嵌入向量。

假设词汇表大小为 $V$，嵌入维度为 $d_{\text{model}}$，则嵌入矩阵 $E \in \mathbb{R}^{V \times d_{\text{model}}}$ 将符号索引 $i$ 映射为向量：
$$
\text{Embedding}(i) = E_i
$$

#### **位置编码（Positional Encoding）**
由于Transformer不使用RNN或CNN，它没有显式的序列顺序信息，因此需要通过位置编码为输入序列添加位置信息。

- **输入**：嵌入向量序列。
- **输出**：带有位置信息的嵌入向量序列。

位置编码利用正弦和余弦函数的不同频率为序列中的每个位置生成唯一的编码：
$$
PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{\frac{2i}{d_{\text{model}}}}}\right)
$$
$$
PE_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{\frac{2i}{d_{\text{model}}}}}\right)
$$
其中，$pos$ 是位置索引，$i$ 是维度索引，$d_\text{model}$ 是维度索引。将位置编码加到嵌入向量上：
$$
X = \text{Embedding} + PE
$$

假设有一个简单的中文句子：“你好吗”。将这个句子转换为词嵌入，并添加位置编码。设定`d_model`（模型的维度）为512，并且每个词已经被转换成了512维的词嵌入向量。

对于第一个词“你”，它的位置是0，所以：

- $PE_{(0, 0)} = \sin\left(\frac{0}{10000^{0/8}}\right)$，$PE_{(0, 1)} = \cos\left(\frac{0}{10000^{0/8}}\right)$，$PE_{(0, 2)} = \sin\left(\frac{0}{10000^{2/8}}\right)$，$PE_{(0, 3)} = \cos\left(\frac{0}{10000^{2/8}}\right)$，...

- 以此类推，直到填满整个512维的位置编码向量。

对于第二个词“好”，它的位置是1，所以：
- $PE_{(1, 0)} = \sin\left(\frac{1}{10000^{0/8}}\right)$，$PE_{(1, 1)} = \cos\left(\frac{1}{10000^{0/8}}\right)$，$PE_{(1, 2)} = \sin\left(\frac{1}{10000^{2/8}}\right)$，...
- 以此类推，直到填满整个512维的位置编码向量。

对于第三个词“吗”，它的位置是2，那么：
- $PE_{(2, 0)} = \sin\left(\frac{2}{10000^{0/512}}\right)$,$PE_{(2, 1)} = \cos\left(\frac{2}{10000^{0/512}}\right)$，$PE_{(2, 2)} = \sin\left(\frac{2}{10000^{2/8}}\right)$，...


#### **注意力机制（Attention）**
假设要将英文句子 "The cat sat on the mat." 翻译成法语 "Le chat était assis sur le tapis." 我们可以使用基于RNN（循环神经网络）的序列到序列（Seq2Seq）模型来进行这个翻译任务。然而，传统的Seq2Seq模型有一个限制，即编码器（Encoder）需要将整个源句子的信息压缩成一个固定长度的向量表示，然后解码器（Decoder）根据这个单一的向量生成目标语言的句子。这种方法对于较长的句子来说效果并不理想，因为过多的信息被压缩进了一个向量，可能会导致信息丢失。

为了解决这个问题，Bahdanau等人在2015年提出了注意力机制。

**缩放点积注意力（Scaled Dot-Product Attention）**
- **输入**：查询矩阵 $Q$、键矩阵 $K$、值矩阵 $V$。
- **输出**：加权后的值向量。

计算注意力分数：
$$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right) V
$$
其中，$d_k$ 是键向量的维度，$\sqrt{d_k}$ 用于缩放点积，防止梯度消失。


为了将注意力机制的计算过程用向量或矩阵表示，我们可以以一个简化版的例子来说明。假设我们有一个简单的输入序列，每个词被嵌入到一个二维空间中（为了便于演示，实际应用中维度通常会更大）。我们的目标是通过注意力机制计算出针对每个词的上下文向量。

**Attention计算过程示例**

假设有三个词的嵌入向量组成的输入矩阵 $X$，每个词的嵌入向量为2维：

$$ X = \begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix} = \begin{bmatrix} 1 & 2 \\ 3 & 4 \\ 5 & 6 \end{bmatrix} $$

**（1）计算Q, K, V**

使用不同的权重矩阵 $W^Q$, $W^K$, 和 $W^V$ 来计算查询（Query）、键（Key）和值（Value）向量。为了简化计算，假定这些权重矩阵也是2维的。

$$ W^Q = \begin{bmatrix} 0.5 & 0.7 \\ 0.8 & 0.9 \end{bmatrix}, \quad W^K = \begin{bmatrix} 0.3 & 0.4 \\ 0.6 & 0.5 \end{bmatrix}, \quad W^V = \begin{bmatrix} 0.2 & 0.3 \\ 0.4 & 0.5 \end{bmatrix} $$

通过矩阵乘法得到 Q, K, V：

$$ Q = XW^Q = \begin{bmatrix} 1 & 2 \\ 3 & 4 \\ 5 & 6 \end{bmatrix} \begin{bmatrix} 0.5 & 0.7 \\ 0.8 & 0.9 \end{bmatrix} = \begin{bmatrix} 2.1 & 2.5 \\ 4.7 & 5.7 \\ 7.3 & 8.9 \end{bmatrix} $$

$$ K = XW^K = \begin{bmatrix} 1 & 2 \\ 3 & 4 \\ 5 & 6 \end{bmatrix} \begin{bmatrix} 0.3 & 0.4 \\ 0.6 & 0.5 \end{bmatrix} = \begin{bmatrix} 1.5 & 1.4 \\ 3.3 & 3.2 \\ 5.1 & 5.0 \end{bmatrix} $$

$$ V = XW^V = \begin{bmatrix} 1 & 2 \\ 3 & 4 \\ 5 & 6 \end{bmatrix} \begin{bmatrix} 0.2 & 0.3 \\ 0.4 & 0.5 \end{bmatrix} = \begin{bmatrix} 1.0 & 1.3 \\ 2.2 & 2.9 \\ 3.4 & 4.5 \end{bmatrix} $$
**（2）计算注意力分数**

计算点积相似度并除以键向量维度的平方根（即 $\sqrt{d_k}$），其中 $d_k=2$：

$$ S = \frac{QK^T}{\sqrt{d_k}} $$

$$ S = \frac{\begin{bmatrix} 2.1 & 2.5 \\ 4.7 & 5.7 \\ 7.3 & 8.9 \end{bmatrix} \begin{bmatrix} 1.5 & 3.3 & 5.1 \\ 1.4 & 3.2 & 5.0 \end{bmatrix}}{\sqrt{2}} $$

$$ S = \frac{\begin{bmatrix} 7.7 & 16.5 & 25.3 \\ 17.49 & 36.3 & 55.11 \\ 27.28 & 56.1 & 84.92 \end{bmatrix}}{\sqrt{2}} \approx \begin{bmatrix} 5.44 & 11.67 & 17.90 \\ 12.37 & 25.69 & 39.00 \\ 19.28 & 39.71 & 60.11 \end{bmatrix} $$

**（3）归一化**

对得分进行softmax归一化处理，得到注意力权重矩阵 $A$：

$$ A = \text{softmax}(S) $$

**（4）上下文向量**

最后，利用注意力权重对值矩阵加权求和，得到上下文向量 $C$：

$$ C = AV $$

In [1]:
import torch
import torch.nn.functional as F

# 假设的输入、权重矩阵
X = torch.tensor([[1., 2.], [3., 4.], [5., 6.]])
W_Q = torch.tensor([[0.5, 0.7], [0.8, 0.9]])
W_K = torch.tensor([[0.3, 0.4], [0.6, 0.5]])
W_V = torch.tensor([[0.2, 0.3], [0.4, 0.5]])

# 计算Q, K, V
Q = torch.matmul(X, W_Q)
K = torch.matmul(X, W_K)
V = torch.matmul(X, W_V)

# 计算注意力分数并归一化
scores = torch.matmul(Q, K.t()) / (K.size(1)**0.5)
attention_weights = F.softmax(scores, dim=-1)

# 计算上下文向量
context_vector = torch.matmul(attention_weights, V)

print("Attention Weights:\n", attention_weights)
print("Context Vector:\n", context_vector)

Attention Weights:
 tensor([[8.1903e-06, 2.8578e-03, 9.9713e-01],
        [3.1802e-12, 1.7833e-06, 1.0000e+00],
        [1.2313e-18, 1.1096e-09, 1.0000e+00]])
Context Vector:
 tensor([[3.3966, 4.4954],
        [3.4000, 4.5000],
        [3.4000, 4.5000]])


#### **多头注意力机制（Multi-Head Attention）**
多头注意力通过并行计算多个注意力头，捕捉不同的子空间信息。

- **输入**：查询矩阵 $Q$、键矩阵 $K$、值矩阵 $V$。
- **输出**：多个注意力头的拼接结果。

- **线性变换**：
   对于输入序列 $X$，先通过线性变换生成 $Q$、$K$ 和 $V$ 矩阵。
   
   $$ Q = XW^Q, \quad K = XW^K, \quad V = XW^V $$


- **分割成多个头**：
   将 $Q$、$K$ 和 $V$ 按照指定的头数 $h$ 切分成更小的矩阵，使得每个头都有自己的 $Q_i$、$K_i$ 和 $V_i$ 矩阵，它们的维度通常为 $d_{model}/h$。

- **计算每个头的注意力输出**：
   对于每个头 $i$，使用上述公式计算其注意力输出 $head_i$。
   $$
   \text{head}_h = \text{Attention}(QW_h^Q, KW_h^K, VW_h^V)
   $$

- **合并与最终变换**：
   将所有头的输出拼接起来形成一个新的矩阵，并通过另一个线性变换将其映射回原始维度，即：

   $$ \text{MultiHead}(Q, K, V) = \text{Concat}(head_1, ..., head_h)W^O $$

   其中 $W^O$ 是最终线性变换的权重矩阵。


#### **残差连接与归一化层**
残差连接（Residual Connection）用于缓解梯度消失问题，帮助模型训练更深层的网络。

- **输入**：子层（如多头注意力或 FFN）的输入和输出。
- **输出**：输入与输出的和。

$$
\text{Output} = \text{Layer}(x) + x
$$

归一化层（Layer Normalization）对每个样本的特征进行归一化，加速训练并提高稳定性。

- **输入**：子层的输出。
- **输出**：归一化后的结果。

$$
\text{LayerNorm}(x) = \gamma \cdot \frac{x - \mu}{\sqrt{\sigma^2 + \epsilon}} + \beta
$$
其中，$\mu$ 和 $\sigma^2$ 是均值和方差，$\gamma$ 和 $\beta$ 是可学习的参数，$\epsilon$ 是防止除零的小常数。

#### **前馈全连接层（Feed-Forward Network, FFN）**
前馈全连接层对每个位置的表示进行非线性变换。

- **输入**：多头注意力层的输出。
- **输出**：经过非线性变换的表示。

$$
\text{FFN}(x) = \max(0, xW_1 + b_1) W_2 + b_2
$$
其中，$W_1, W_2$ 是权重矩阵，$b_1, b_2$ 是偏置项。

#### **Softmax 和输出层**
Softmax 用于将模型的输出转换为概率分布。

- **输入**：解码器的最终输出。
- **输出**：目标词汇的概率分布。

$$
\text{Softmax}(z_i) = \frac{e^{z_i}}{\sum_{j=1}^V e^{z_j}}
$$
其中，$z_i$ 是第 $i$ 个词汇的得分，$V$ 是词汇表大小。

## 2.1.2 示例：Bert模型
### Bert

BERT（Bidirectional Encoder Representations from Transformers）是一种基于Transformer架构的预训练语言模型，由Google在2018年提出。BERT的核心思想是通过双向上下文来理解文本，而不是像传统的语言模型那样只从左到右或从右到左进行建模。参见[BERT：Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/pdf/1810.04805)

#### BERT的主要结构
1. **输入表示**：
   - **Token Embeddings**：将输入的单词或子词转换为向量表示。
   - **Segment Embeddings**：用于区分句子对（例如，问答任务中的问题和答案）。如果输入包含两个句子，则需要区分这两个句子。为此，BERT为第一个句子的所有tokens分配一个值0，第二个句子的所有tokens分配一个值1。
   - **Position Embeddings**：表示单词在句子中的位置信息。

2. **Transformer Encoder**：
   - BERT使用了多层Transformer Encoder堆叠而成。
   - 每个Transformer Encoder层包含两个子层：
     - **Multi-Head Self-Attention**：计算输入序列中每个单词与其他单词的相关性。
     - **Feed-Forward Neural Network**：对每个位置的表示进行非线性变换。

3. **输出**：
   - BERT的输出是每个输入token的上下文表示，可以用于各种下游任务（如文本分类、命名实体识别等）。

  ![alt text](images/Bert.jpg)

#### BERT的预训练任务

1. **Masked Language Model (MLM)**：
   - 随机掩盖输入序列中的一些token，然后让模型预测这些被掩盖的token。

2. **Next Sentence Prediction (NSP)**：
   - 给定两个句子，模型预测第二个句子是否是第一个句子的下一句。

#### 实现一个基于BERT的文本分类器

下面是一个使用`transformers`库和`PyTorch`实现基于BERT的文本分类器的示例代码，参见源码bert_cls.py。

1. **数据准备**：
   - 使用`train_test_split`将数据划分为训练集和测试集。
   - 自定义`TextDataset`类来处理文本数据，并将其转换为BERT模型所需的输入格式。

2. **模型加载**：
   - 使用`BertForSequenceClassification`加载预训练的BERT模型，并指定分类的类别数（`num_labels=2`表示二分类）。

3. **训练和评估**：
   - 定义`train_epoch`和`eval_model`函数来分别进行训练和评估。
   - 使用`AdamW`优化器进行模型训练。

4. **设备设置**：
   - 检查是否有可用的GPU，并将模型和数据移动到相应的设备上。

5. **训练循环**：
   - 进行多个epoch的训练，并在每个epoch结束后评估模型在验证集上的表现。

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# 示例数据
'''
texts = ["I love programming", "I hate bugs", "Python is great", "Debugging is frustrating"]
labels = [1, 0, 1, 0]  # 1: Positive, 0: Negative
# 划分训练集和测试集
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)
'''
import pandas as pd
data = {
    'text': [
        'I love programming in Python!',
        'This is the worst experience I have ever had.',
        'Absolutely fantastic! Highly recommend it.',
        'Not good, not bad, just okay.',
        'The movie was thrilling and full of action.',
        'I did not like the plot of the book.',
        'The concert was amazing with great music and energy.',
        'The food at the restaurant was terrible.'
    ],
    'label': [0, 1, 0, 1, 2, 1, 2, 1]  # 假设有3个类别：0 - 正面，1 - 负面，2 - 中性
}

df = pd.DataFrame(data)

# 分割数据集为训练集和测试集
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['text'].tolist(),
    df['label'].tolist(),
    test_size=0.2,
    random_state=42
)


model_path = "bert/bert-base-uncased"
# 加载BERT tokenizer
tokenizer = BertTokenizer.from_pretrained(model_path)

# 自定义Dataset类
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# 创建Dataset和DataLoader
train_dataset = TextDataset(train_texts, train_labels, tokenizer)
test_dataset = TextDataset(test_texts, test_labels, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2)

# 加载预训练的BERT模型
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=3)

# 设置优化器
optimizer = AdamW(model.parameters(), lr=2e-5)

# 训练函数
def train_epoch(model, data_loader, optimizer, device):
    model = model.train()
    total_loss = 0
    correct_predictions = 0

    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        _, preds = torch.max(logits, dim=1)
        correct_predictions += torch.sum(preds == labels)
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    return correct_predictions.double() / len(data_loader.dataset), total_loss / len(data_loader)

# 评估函数
def eval_model(model, data_loader, device):
    model = model.eval()
    correct_predictions = 0
    total_loss = 0

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            _, preds = torch.max(logits, dim=1)
            correct_predictions += torch.sum(preds == labels)
            total_loss += loss.item()

    return correct_predictions.double() / len(data_loader.dataset), total_loss / len(data_loader)

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 记录损失
loss_history = []
# 训练和评估
EPOCHS = 5
for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    train_acc, train_loss = train_epoch(model, train_loader, optimizer, device)
    loss_history.append(train_loss)
    print(f'Train loss: {train_loss}, Train accuracy: {train_acc}')

    val_acc, val_loss = eval_model(model, test_loader, device)
    print(f'Validation loss: {val_loss}, Validation accuracy: {val_acc}')


# 绘制损失曲线
plt.plot(loss_history)
plt.grid(True)
plt.xlabel("epoch")
plt.ylabel("Loss")
plt.title("Training Loss")
plt.show()